In [1]:
import pandas as pd
import numpy as np

In [2]:
test = pd.read_csv( 'application_test.csv')
train = pd.read_csv('application_train.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')
credit_card_balance = pd.read_csv('credit_card_balance.csv')
POS_CASH_balance = pd.read_csv('POS_CASH_balance.csv')
previous_application = pd.read_csv('previous_application.csv')
sample_submission = pd.read_csv('sample_submission.csv')
HomeCredit_columns_description = pd.read_csv('HomeCredit_columns_description.csv', encoding='ISO-8859-1')
installments_payments = pd.read_csv('installments_payments.csv')
bureau=pd.read_csv('bureau.csv')

In [16]:
sample_submission

,SK_ID_CURR,TARGET
0,100001,0.5
1,100005,0.5
2,100013,0.5
3,100028,0.5
4,100038,0.5
...,...,...
48739,456221,0.5
48740,456222,0.5
48741,456223,0.5
48742,456224,0.5


In [3]:
train.shape

(307511, 122)

In [4]:
test.shape

(48744, 121)

# Fusionner avec bureau et bureau_balance

In [5]:

def merge_with_bureau_bureau_balance(train):
    # Filtrer bureau pour les entrées correspondant à SK_ID_CURR dans train ou test
    all_ids = pd.concat([train['SK_ID_CURR'], test['SK_ID_CURR']]).unique()
    bureau_filtered = bureau[bureau['SK_ID_CURR'].isin(all_ids)]

    # Filtrer bureau_balance pour les entrées correspondant à SK_ID_BUREAU dans bureau_filtered
    bureau_balance_filtered = bureau_balance[bureau_balance['SK_ID_BUREAU'].isin(bureau_filtered['SK_ID_BUREAU'])]

    # Aggregations pour bureau_balance
    agg_bureau_balance = bureau_balance_filtered.groupby('SK_ID_BUREAU').agg({
        'MONTHS_BALANCE': ['mean']
    })
    agg_bureau_balance.columns = ['BB_' + '_'.join(col).strip() for col in agg_bureau_balance.columns.values]

    # Fusionner les agrégations avec bureau_filtered
    bureau_filtered = bureau_filtered.merge(agg_bureau_balance, on='SK_ID_BUREAU', how='left')

    # Créer des dummies pour CREDIT_ACTIVE
    credit_active_dummies = pd.get_dummies(bureau_filtered['CREDIT_ACTIVE'], prefix='ca')
    credit_active_dummies['SK_ID_CURR'] = bureau_filtered['SK_ID_CURR']
    credit_active_counts = credit_active_dummies.groupby('SK_ID_CURR').sum()

    # Créer des dummies pour CREDIT_TYPE
    credit_type_dummies = pd.get_dummies(bureau_filtered['CREDIT_TYPE'], prefix='ct')
    credit_type_dummies['SK_ID_CURR'] = bureau_filtered['SK_ID_CURR']
    credit_type_counts = credit_type_dummies.groupby('SK_ID_CURR').sum()

    # Devise la plus fréquente
    most_frequent_currency = bureau_filtered.groupby('SK_ID_CURR')['CREDIT_CURRENCY'].agg(lambda x: x.value_counts().index[0])

    # Aggregations pour bureau
    agg_bureau = bureau_filtered.groupby('SK_ID_CURR').agg({
        'DAYS_CREDIT': ['min', 'max', 'mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'sum'],
        'DAYS_CREDIT_ENDDATE': ['max'],
        'DAYS_ENDDATE_FACT': ['mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean', 'max'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'AMT_CREDIT_SUM': ['sum'],
        'AMT_CREDIT_SUM_DEBT': ['sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'AMT_ANNUITY': ['mean', 'sum'],
    })
    agg_bureau.columns = ['BUREAU_' + '_'.join(col).strip() for col in agg_bureau.columns.values]

    # Fusionner toutes les agrégations avec train et test
    train = train.merge(agg_bureau, on='SK_ID_CURR', how='left')
    train = train.merge(credit_active_counts, on='SK_ID_CURR', how='left')
    train = train.merge(credit_type_counts, on='SK_ID_CURR', how='left')
    train['MOST_FREQ_CURRENCY'] = train['SK_ID_CURR'].map(most_frequent_currency)

    return train


# Fusionner avec POS_CASH_balance

In [6]:

def merge_with_pos_cash_balance(train):
    # Filtrer POS_CASH_balance pour les entrées correspondant à SK_ID_CURR dans train ou test
    all_ids = pd.concat([train['SK_ID_CURR'], test['SK_ID_CURR']]).unique()
    pos_cash_filtered = POS_CASH_balance[POS_CASH_balance['SK_ID_CURR'].isin(all_ids)]

    # Aggregations pour POS_CASH_balance
    agg_pos_cash = pos_cash_filtered.groupby('SK_ID_CURR').agg({
        'MONTHS_BALANCE': ['count', 'min', 'max'],
        'CNT_INSTALMENT': ['sum'],
        'CNT_INSTALMENT_FUTURE': ['sum'],
        'SK_DPD': ['max', 'sum'],
        'SK_DPD_DEF': ['max', 'sum']
    })

    # Renommer les colonnes
    agg_pos_cash.columns = ['PCB_' + '_'.join(col).strip() for col in agg_pos_cash.columns.values]

    # Fusionner avec train et test
    train_merged = train.merge(agg_pos_cash, on='SK_ID_CURR', how='left')
    
    return train_merged


# Fusionner avec credit_card_balance

In [7]:

def merge_with_credit_card_balance(train):
    # Filtrer credit_card_balance pour les entrées correspondant à SK_ID_CURR dans train ou test
    all_ids = pd.concat([train['SK_ID_CURR'], test['SK_ID_CURR']]).unique()
    credit_card_filtered = credit_card_balance[credit_card_balance['SK_ID_CURR'].isin(all_ids)]

    # Aggregations pour credit_card_balance
    agg_credit_card = credit_card_filtered.groupby('SK_ID_CURR').agg({
        'AMT_BALANCE': ['sum'],
        'AMT_CREDIT_LIMIT_ACTUAL': ['sum'],
        'AMT_DRAWINGS_ATM_CURRENT': ['sum'],
        'AMT_DRAWINGS_CURRENT': ['sum'],
        'AMT_PAYMENT_CURRENT': ['sum'],
        'CNT_DRAWINGS_CURRENT': ['sum'],
        'SK_DPD': ['max', 'sum'],
        'SK_DPD_DEF': ['max', 'sum']
    })

    # Renommer les colonnes
    agg_credit_card.columns = ['CCB_' + '_'.join(col).strip() for col in agg_credit_card.columns.values]

    # Fusionner avec train et test
    train_merged = train.merge(agg_credit_card, on='SK_ID_CURR', how='left')

    return train_merged


# Fusionner avec previous_application

In [8]:

def merge_with_previous_application(train):
    # Filtrer previous_application pour les entrées correspondant à SK_ID_CURR dans train ou test
    all_ids = pd.concat([train['SK_ID_CURR'], test['SK_ID_CURR']]).unique()
    previous_filtered = previous_application[previous_application['SK_ID_CURR'].isin(all_ids)]

    # Aggregations pour previous_application
    agg_previous = previous_filtered.groupby('SK_ID_CURR').agg({
        'AMT_ANNUITY': ['sum'],
        'AMT_APPLICATION': ['sum'],
        'AMT_CREDIT': ['sum'],
        'AMT_DOWN_PAYMENT': ['sum'],
        'AMT_GOODS_PRICE': ['sum'],
        'RATE_DOWN_PAYMENT': ['mean'],
        'DAYS_DECISION': ['mean'],
        'CNT_PAYMENT': ['sum']
    })

    # Renommer les colonnes
    agg_previous.columns = ['PA_' + '_'.join(col).strip() for col in agg_previous.columns.values]

    # Fusionner avec train et test
    train_merged = train.merge(agg_previous, on='SK_ID_CURR', how='left')

    return train_merged


# Fusionner avec installments_payments

In [9]:

def merge_with_installments_payments(train):
    # Filtrer installments_payments pour les entrées correspondant à SK_ID_CURR dans train ou test
    all_ids = pd.concat([train['SK_ID_CURR'], test['SK_ID_CURR']]).unique()
    installments_filtered = installments_payments[installments_payments['SK_ID_CURR'].isin(all_ids)]

    # Aggregations pour installments_payments
    agg_installments = installments_filtered.groupby('SK_ID_CURR').agg({
        'NUM_INSTALMENT_VERSION': ['mean'],
        'NUM_INSTALMENT_NUMBER': ['max'],
        'DAYS_INSTALMENT': ['mean'],
        'DAYS_ENTRY_PAYMENT': ['mean'],
        'AMT_INSTALMENT': ['sum'],
        'AMT_PAYMENT': ['sum']
    })

    # Renommer les colonnes
    agg_installments.columns = ['INST_' + '_'.join(col).strip() for col in agg_installments.columns.values]

    # Fusionner avec train et test
    train_merged = train.merge(agg_installments, on='SK_ID_CURR', how='left')

    return train_merged


# Fusionner avec tous les frames

In [10]:
def merge_with(train):
    train1 = train.copy()
    train1 = merge_with_bureau_bureau_balance(train1)
    train1 = merge_with_pos_cash_balance(train1)
    train1 = merge_with_credit_card_balance(train1)
    train1 = merge_with_previous_application(train1)
    train1 = merge_with_installments_payments(train1)
    return train1

In [11]:
train_merged=merge_with(train)

In [12]:
print(train_merged.shape)

(307511, 191)


In [13]:
test_merged=merge_with(test)

In [14]:
print(test_merged.shape)


(48744, 187)


In [15]:
train_merged.to_csv('train_merged.csv')
train_merged.to_csv('test_merged.csv')